In [17]:
import random
import itertools
from tqdm.notebook import tqdm

In [12]:
def draw_board(board):
  print("-----------------")
  for row in board:
    print(' '.join(map(str, row)))
  print("-----------------")

def compress_board(board):
    W, H = len(board), len(board[0])
    seq = []
    for x in range(W):
        for y in range(H):
            if board[x][y] != 0:
                seq.append(board[x][y])
    jseq = ','.join(map(str, seq))
    print("buildBoard(%d, []int{%s})" % (W, jseq))

In [9]:
def s10req(n, s, m):
    if n == 0 and s == 0:
        yield []
        return
    if m > s or n == 0 or s < 0:
        return
    for p in range(m, min(9, s) + 1):
        for seq in s10req(n - 1, s - p, p):
            yield [p] + seq
    
sum_10_seq = {
    i: list(s10req(i, 10, 1)) for i in range(2, 5)
}   

In [22]:
def find_glades(board):
    W, H = len(board), len(board[0])
    # TODO: do better
    def _is_glade(lx, ly, rx, ry):
        if lx == rx and ly == ry:
            return False
        for x in range(lx, rx+1):
            for y in range(ly, ry+1):
                if board[x][y] != 0:
                    return False
        return True
    
    for lx in range(W):
        for ly in range(H):
            for rx in range(lx, W):
                for ry in range(ly, H):
                    if _is_glade(lx, ly, rx, ry):
                        yield (lx, ly, rx, ry)


MAX_NUMS_PER_GLADE = 4
def make_board(w, h, seed, print_board=False):
    random.seed(seed)
    board = [[0 for _ in range(h)] for _ in range(w)]
    
    def _isolated(x, y):
        if board[x][y] != 0:
            return False
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = x+dx, y+dy
            if 0<= nx < w and 0 <= ny < h and board[nx][ny] == 0:
                return False
        return True

    def _rec():
        if all(board[x][y] != 0 for x in range(w) for y in range(h)):
            return True
        if any(_isolated(x, y) for x in range(w) for y in range(h)):
            return False
        glades = list(find_glades(board))      
        random.shuffle(glades)

        for lx, ly, rx, ry in glades:
          gw, gh = rx-lx+1, ry-ly+1
          assert gw*gh >= 2
          all_xy = [(x, y) for x in range(lx, rx+1) for y in range(ly, ry+1)]

          nns = list(reversed(range(2, min(MAX_NUMS_PER_GLADE, gw*gh)+1)))
          
          for nn in nns:
            nums = random.sample(sum_10_seq[nn], 1)[0][:]
            random.shuffle(nums)
            for nxy in itertools.combinations(all_xy, nn):
                for n, (x, y) in zip(nums, nxy):
                    board[x][y] = n
                if _rec():
                    if print_board:
                        dbgboard = [[0 for _ in range(h)] for _ in range(w)]
                        for x, y in nxy:
                            dbgboard[x][y] = board[x][y]
                        draw_board(dbgboard)
                    return True
                for x, y in nxy:
                    board[x][y] = 0
        return False
    
    if not _rec():
        return None
    return board    


[[5, 5, 3, 1, 3],
 [5, 2, 3, 4, 6],
 [1, 6, 1, 1, 1],
 [1, 2, 7, 5, 6],
 [3, 3, 7, 5, 4]]

In [23]:
BOARDS = {}
for seed in tqdm(range(0, 30)):
    board = make_board(5, 5, seed, print_board=False)
    if board is not None:
        BOARDS[seed] = board

for board in BOARDS.values():
    compress_board(board)

  0%|          | 0/30 [00:00<?, ?it/s]

buildBoard(5, []int{6,3,2,2,3,4,3,2,4,1,1,7,1,3,6,1,6,4,7,2,3,1,3,3,2})
buildBoard(5, []int{5,1,1,1,6,5,7,1,1,2,5,4,7,1,2,5,6,4,2,5,8,2,2,2,5})
buildBoard(5, []int{8,2,2,1,9,3,1,2,5,5,3,4,5,4,4,4,5,1,1,1,7,3,2,1,7})
buildBoard(5, []int{2,5,7,3,7,8,5,3,9,1,2,6,4,7,3,8,3,7,4,6,5,5,1,2,7})
buildBoard(5, []int{5,7,2,2,6,5,3,2,7,1,1,1,1,1,9,7,6,1,2,7,6,4,4,2,8})
buildBoard(5, []int{1,7,1,1,2,4,4,5,1,8,6,2,2,2,4,2,5,1,2,2,9,1,4,6,8})
buildBoard(5, []int{2,6,2,1,9,2,4,2,2,2,2,1,4,3,8,3,1,3,5,1,7,2,8,8,2})
buildBoard(5, []int{5,3,5,1,7,5,1,1,1,1,3,3,1,2,7,1,3,2,5,2,3,2,2,3,1})
buildBoard(5, []int{2,8,9,1,4,7,1,1,1,1,3,7,3,7,1,1,3,1,5,4,6,4,4,2,4})
buildBoard(5, []int{6,3,7,3,5,4,4,5,5,2,3,3,2,1,2,3,7,5,2,8,4,4,2,3,7})
buildBoard(5, []int{1,4,1,4,6,2,2,4,2,4,5,5,2,4,6,8,2,2,7,1,5,5,6,1,1})
buildBoard(5, []int{3,3,1,3,1,3,8,2,3,3,1,7,2,5,5,5,2,8,2,2,5,4,6,2,4})
buildBoard(5, []int{1,1,8,6,4,2,3,2,3,1,1,7,3,7,1,8,1,7,9,2,1,2,3,1,6})
buildBoard(5, []int{4,4,4,3,3,2,4,2,1,3,4,2,2,6,1,5,9,1,1,3,5,1,